In [1]:
import time
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from skimage import io
from scipy.sparse import coo_matrix

In [2]:
start = time.time()

In [3]:
%%time
#importation des données
articles = pd.read_csv(r'C:\Users\Stagiaire\Downloads\Timsoft\H&M\articles.csv/articles.csv',nrows=10000)
users = pd.read_csv(r'C:\Users\Stagiaire\Downloads\Timsoft\H&M\customers.csv/customers.csv')
df = pd.read_csv(r'C:\Users\Stagiaire\Downloads\Timsoft\H&M\transactions_train.csv/transactions.csv')

Wall time: 33.8 s


In [4]:
feature_subset = ['product_group_name', 
       'graphical_appearance_name', 'colour_group_name',
       'perceived_colour_value_name',
       'perceived_colour_master_name',
       'department_name', 'index_name',
       'index_group_name', 'section_name',
       'garment_group_name']

In [5]:
features = feature_subset
df2=articles[['article_id']+features]
dummies_df2=pd.get_dummies(df2,columns=features)

In [6]:
#construction de la matrice Features - Items 
item_feature = dummies_df2.drop_duplicates(subset='article_id')
item_feature = item_feature.set_index('article_id')

In [7]:
#creation de la matrice de similarité cosinus
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(item_feature, item_feature)
indices = pd.Series(item_feature.index)
def recommendations(title, cosine_sim = cos_sim):
    recommended_articles = []
    index = indices[indices == title].index[0]
    similarity_scores = pd.Series(cosine_sim[index]).sort_values(ascending = False)
    top_10_articles = list(similarity_scores.iloc[1:11].index)
    for i in top_10_articles:
        recommended_articles.append(list(item_feature.index)[i])
    return recommended_articles

In [8]:
%%time
#creation du tableau d'article et similarité, 10 articles sont proposées
df3 = pd.DataFrame( columns =['article_id', 'Article Similaire'])
for i in range(item_feature.shape[0]):
    art = item_feature.index[i]
    df3.loc[i]=[art,recommendations(art)]

Wall time: 1min 33s


In [9]:
%%time
df3.to_csv('Predictionhm.csv')

Wall time: 64.9 ms


In [10]:
end = time.time()
print(end - start)

129.74298858642578


In [ ]:
#création de la vérification 
newdata=pd.DataFrame(columns=articles.columns)
def Rechercher_caracteristiques(num,newdata):
    for j in range(articles.shape[0]):
        if (articles.iloc[j].article_id)==(num):
            k=j
            break
    newdata=pd.DataFrame(columns=articles.columns)
    L = articles.loc[k]
    newdata=newdata.append(L)
    return newdata.T

In [ ]:
indices = pd.Series(df3.index, index=df3['article_id']).drop_duplicates()
indices2=pd.Series(articles['article_id'])
le_df2=pd.DataFrame(columns=articles.columns)
def construction(articleid,le_df): 
    idx = indices[articleid]
    LI=df3.loc[idx]["Article Similaire"]
    for l in LI:
        index2=indices2[indices2 == l].index[0]
        h=index2
        O = articles.loc[h]
        le_df=le_df.append(O)
    return le_df.T

In [ ]:
%%time
construction(537631002,le_df2)

In [ ]:
indices = pd.Series(item_feature.index)
indices2=pd.Series(articles['article_id'])
le_df=pd.DataFrame( columns =['article_id', 'Caractéristiques', 'Similarité'])
Features_a_inclure = ['prod_name', 'index_name', 'section_name', 'product_group_name', 'product_type_name', 'garment_group_name', 'detail_desc', 'colour_group_name']
def recommendations_bis(article_id, le_df, cosine_sim = cos_sim):
    #construction de l'index,articles et taux de similarité
    recommended_articles = []
    index = indices[indices == article_id].index[0]
    similarity_scores = pd.Series(cosine_sim[index]).sort_values(ascending = False)
    top_12_articles_scores = list(similarity_scores.iloc[1:13])
    top_12_articles_index= list(similarity_scores.iloc[1:13].index)
    for i in top_12_articles_index:
        recommended_articles.append(list(item_feature.index)[i])
    #construction du dataframe final
    for l in recommended_articles:
        index2=indices2[indices2 == l].index[0]
        h=index2
        C=articles[Features_a_inclure].iloc[h].values
        D=recommended_articles.index(l)
        O = [articles.iloc[index2].prod_name,C,top_12_articles_scores[D]]
        le_df.loc[D] = O
    return le_df

In [ ]:
%%time
df7=recommendations_bis(537631002,le_df,cosine_sim = cos_sim)
df7

In [ ]:
%%time
df8=Rechercher_caracteristiques(673677001,newdata)
df8

In [ ]:
df7.to_csv('df8.csv')

In [ ]:
def plot_prev(prev_items):
    fig = plt.figure(figsize=(20, 10))
    for item, i in zip(prev_items, range(1, len(prev_items)+1)):
        item = '0' + str(item)
        sub = item[:3]
        image = path + "/"+ sub + "/"+ item +".jpg"
        image = plt.imread(image)
        fig.add_subplot(1, k, i)
        plt.imshow(image)

In [ ]:
def plot_rcmnd(rcmnds):
    fig = plt.figure(figsize=(20, 10))
    for item, i in zip(rcmnds, range(1, k+1)):
        item = '0' + str(item)
        sub = item[:3]
        image = path + "/"+ sub + "/"+ item +".jpg"
        image = plt.imread(image)
        fig.add_subplot(1, k, i)
        plt.imshow(image)

In [ ]:
k=10
rcmnds=df3.iloc[9983]["Article Similaire"]
path = "F:\images"

In [ ]:
plot_prev(['537631002'])

In [ ]:
plot_rcmnd(rcmnds)

In [ ]:
rcmnds